# Inicjalizacja kontekstu użytkownika

Cel: izolacja zasobów dla każdego uczestnika. Tworzone są schematy per użytkownik w katalogu `training_catalog`: `<user_slug>_bronze`, `<user_slug>_silver`, `<user_slug>_gold`. Uruchom tę komórkę przed dalszymi krokami w innych notebookach.

## 📁 Architektura danych w szkoleniu

Szkolenie używa **dwóch podejść do przechowywania danych** w zależności od dnia:

### Dzień 1-2: Lokalne pliki → Delta Tables
- **Źródło**: Folder `dataset/` (CSV, JSON, Parquet)
- **Zmienna**: `DATASET_BASE_PATH` 
- **Cel**: Załadowanie do Delta tables w schematach Bronze/Silver/Gold
- **Przykład**: `spark.read.csv(f"{DATASET_BASE_PATH}/customers/customers.csv")`

### Dzień 3: Unity Catalog Volumes
- **Źródło**: Unity Catalog Volumes (`/Volumes/training_catalog/default/kion_datasets`)
- **Zmienna**: `VOLUMES_BASE_PATH` (opcjonalne dla Dzień 3)
- **Cel**: Demonstracja zaawansowanych feature'ów UC (DLT, Lakeflow, Governance)
- **Przykład**: `spark.read.csv("/Volumes/training_catalog/default/kion_datasets/customers.csv")`

### Dlaczego dwa podejścia?
- **Progresja edukacyjna**: Od prostych plików (Dzień 1) → Delta Lake (Dzień 2) → Unity Catalog (Dzień 3)
- **Real-world scenarios**: W produkcji często używa się Volumes dla managed data access w UC
- **Best practices**: Dzień 3 pokazuje, jak zarządzać danymi w enterprise environment

---

In [0]:
# === Konfiguracja katalogów i schematów ===
CATALOG = "training_catalog"

# Pobierz aktualnego użytkownika (dla środowiska produkcyjnego)
# raw_user = spark.sql("SELECT current_user()").first()[0]
raw_user = "trainer"  # Dla środowiska szkoleniowego

import re
user_slug = re.sub(
    r'[^a-z0-9]', 
    '_', 
    raw_user.lower()
)

print(f"User slug: {user_slug}")

# Schematy per użytkownik (izolacja zasobów)
BRONZE_SCHEMA = f"{user_slug}_bronze"
SILVER_SCHEMA = f"{user_slug}_silver"
GOLD_SCHEMA   = f"{user_slug}_gold"

# === Ścieżka do datasetów ===
# Dla lokalnego środowiska - ścieżka do folderu dataset w tym repo
import os
DATASET_BASE_PATH = os.path.abspath("../dataset")

# Dla Databricks środowiska - Unity Catalog Volume:
# DATASET_BASE_PATH = "/Volumes/training_catalog/default/kion_datasets"


print(f"Dataset base path: {DATASET_BASE_PATH}")

# === Tworzenie katalogów i schematów (tylko w Databricks) ===
try:
    spark.sql(f'USE CATALOG {CATALOG}')
    
    for s in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
        spark.sql(f'CREATE SCHEMA IF NOT EXISTS {CATALOG}.{s}')
    
    spark.sql(f'USE SCHEMA {BRONZE_SCHEMA}')
    print("✓ Unity Catalog schemas created successfully")
except Exception as e:
    print(f"⚠ Unity Catalog not available or not configured: {e}")
    print("Continuing with default database...")

# === Wyświetl informacje o katalogu ===
print("\n=== Informacje o katalogu ===")
display(
    spark.sql(f"DESCRIBE CATALOG EXTENDED {CATALOG}")
)

# === Podsumowanie konfiguracji ===
print("\n=== Podsumowanie konfiguracji użytkownika ===")
display(
    spark.createDataFrame(
        [
            (
                raw_user, 
                f'{CATALOG}.{BRONZE_SCHEMA}', 
                f'{CATALOG}.{SILVER_SCHEMA}', 
                f'{CATALOG}.{GOLD_SCHEMA}',
                DATASET_BASE_PATH
            )
        ],
        ['user', 'bronze_schema', 'silver_schema', 'gold_schema', 'dataset_path']
    )
)

print("\n✓ Inicjalizacja zakończona pomyślnie!")
print(f"✓ Użytkownik: {raw_user}")
print(f"✓ Katalog: {CATALOG}")
print(f"✓ Schematy: {BRONZE_SCHEMA}, {SILVER_SCHEMA}, {GOLD_SCHEMA}")
print(f"✓ Ścieżka do danych: {DATASET_BASE_PATH}")


## 🔧 Funkcje pomocnicze (opcjonalne - tylko dla Dzień 3)

Poniższa funkcja kopiuje dane z lokalnego folderu `dataset/` do Unity Catalog Volumes.  
**Użyj tylko jeśli pracujesz na Databricks i chcesz przygotować dane dla Dzień 3.**

In [ ]:
def copy_dataset_to_volumes():
    """
    Kopiuje dane z lokalnego folderu dataset/ do Unity Catalog Volumes.
    Używane tylko w środowisku Databricks dla przygotowania Dzień 3.
    """
    import shutil
    from pathlib import Path
    
    print("=== Kopiowanie danych do Unity Catalog Volumes ===")
    print(f"Źródło: {DATASET_BASE_PATH}")
    print(f"Cel: {VOLUMES_BASE_PATH}")
    
    try:
        # Sprawdź, czy Volume istnieje
        dbutils.fs.ls(VOLUMES_BASE_PATH)
        print("✓ Volume dostępny")
        
        # Kopiuj każdy folder z dataset/
        folders_to_copy = ['customers', 'orders', 'products']
        
        for folder in folders_to_copy:
            source = os.path.join(DATASET_BASE_PATH, folder)
            target = f"{VOLUMES_BASE_PATH}/{folder}"
            
            if os.path.exists(source):
                print(f"  Kopiowanie: {folder}/")
                
                # Utwórz folder w Volumes jeśli nie istnieje
                try:
                    dbutils.fs.mkdirs(target)
                except:
                    pass
                
                # Kopiuj pliki
                for file in os.listdir(source):
                    source_file = os.path.join(source, file)
                    target_file = f"{target}/{file}"
                    
                    if os.path.isfile(source_file):
                        dbutils.fs.cp(f"file:{source_file}", target_file, recurse=False)
                        print(f"    ✓ {file}")
            else:
                print(f"  ⚠ Folder {folder}/ nie istnieje w dataset/")
        
        print("\n✓ Kopiowanie zakończone pomyślnie!")
        print(f"Dane dostępne w: {VOLUMES_BASE_PATH}")
        
        return True
        
    except Exception as e:
        print(f"\n✗ Błąd kopiowania: {e}")
        print("Upewnij się, że:")
        print("  1. Unity Catalog jest włączony")
        print("  2. Volume został utworzony: CREATE VOLUME IF NOT EXISTS training_catalog.default.kion_datasets")
        print("  3. Masz uprawnienia do zapisu w Volume")
        return False

# Odkomentuj poniższą linię, aby skopiować dane do Volumes (tylko raz, przed Dzień 3)
# copy_dataset_to_volumes()

## 🔍 Funkcje diagnostyczne

Pomocne funkcje do sprawdzania stanu środowiska i danych.

In [ ]:
def show_environment_info():
    """Wyświetla pełne informacje o konfiguracji środowiska"""
    print("=" * 70)
    print("📊 INFORMACJE O ŚRODOWISKU SZKOLENIOWYM")
    print("=" * 70)
    
    print(f"\n👤 Użytkownik: {raw_user}")
    print(f"   User slug: {user_slug}")
    
    print(f"\n📁 Schematy:")
    print(f"   Bronze: {CATALOG}.{BRONZE_SCHEMA}")
    print(f"   Silver: {CATALOG}.{SILVER_SCHEMA}")
    print(f"   Gold:   {CATALOG}.{GOLD_SCHEMA}")
    
    print(f"\n💾 Ścieżki do danych:")
    print(f"   Dataset (Dzień 1-2): {DATASET_BASE_PATH}")
    print(f"   Volumes (Dzień 3):   {VOLUMES_BASE_PATH}")
    
    print(f"\n⚙️ Spark:")
    print(f"   Spark version: {spark.version}")
    try:
        print(f"   Databricks Runtime: {spark.conf.get('spark.databricks.clusterUsageTags.sparkVersion')}")
    except:
        print(f"   Databricks Runtime: N/A (local environment)")
    
    print(f"\n🗄️ Unity Catalog:")
    try:
        current_catalog = spark.sql("SELECT current_catalog()").first()[0]
        current_schema = spark.sql("SELECT current_schema()").first()[0]
        print(f"   Current catalog: {current_catalog}")
        print(f"   Current schema: {current_schema}")
        print(f"   Status: ✓ Aktywny")
    except:
        print(f"   Status: ✗ Niedostępny")
    
    print("\n" + "=" * 70)

def show_tables_summary():
    """Wyświetla podsumowanie tabel utworzonych w schematach"""
    print("=" * 70)
    print("📋 PODSUMOWANIE TABEL W SCHEMATACH")
    print("=" * 70)
    
    for schema_name, schema_var in [
        ("Bronze", BRONZE_SCHEMA),
        ("Silver", SILVER_SCHEMA),
        ("Gold", GOLD_SCHEMA)
    ]:
        print(f"\n{schema_name} Layer ({CATALOG}.{schema_var}):")
        try:
            tables = spark.sql(f"SHOW TABLES IN {CATALOG}.{schema_var}")
            table_count = tables.count()
            
            if table_count > 0:
                print(f"  ✓ {table_count} tabel(i):")
                for row in tables.collect():
                    table_type = "VIEW" if row.isTemporary else "TABLE"
                    print(f"    - {row.tableName} ({table_type})")
            else:
                print(f"  ⚠ Brak tabel (schemat pusty)")
        except Exception as e:
            print(f"  ✗ Nie można odczytać tabel: {e}")
    
    print("\n" + "=" * 70)

def show_dataset_statistics():
    """Wyświetla statystyki plików w folderze dataset/"""
    print("=" * 70)
    print("📈 STATYSTYKI DANYCH ŹRÓDŁOWYCH")
    print("=" * 70)
    
    folders = ['customers', 'orders', 'products']
    
    for folder in folders:
        folder_path = os.path.join(DATASET_BASE_PATH, folder)
        print(f"\n📂 {folder}/")
        
        if os.path.exists(folder_path):
            files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
            print(f"  Liczba plików: {len(files)}")
            
            total_size = 0
            for file in files:
                file_path = os.path.join(folder_path, file)
                size = os.path.getsize(file_path)
                total_size += size
                size_mb = size / (1024 * 1024)
                print(f"    - {file}: {size_mb:.2f} MB")
            
            total_mb = total_size / (1024 * 1024)
            print(f"  Łączny rozmiar: {total_mb:.2f} MB")
        else:
            print(f"  ✗ Folder nie istnieje")
    
    print("\n" + "=" * 70)

# Przykład użycia (odkomentuj aby uruchomić):
# show_environment_info()
# show_tables_summary()
# show_dataset_statistics()